In [1]:
from konlpy.tag import Okt
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
import numpy as np
from nltk.corpus import stopwords
from bs4 import BeautifulSoup
import re
from tqdm import tqdm
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.models import load_model

In [2]:
df=pd.read_csv('movie_data.csv')
df

,영화명,평점,리뷰
0,범죄도시2,10,NaN
1,범죄도시2,10,내누군지아니 나 장첸이야~!! ㅋ
2,범죄도시2,10,시원시원한 타격감 ㅋㅋㅋ 청량하다 마동석
3,범죄도시2,10,최고의 마동석표 웰메이드 액션 코믹 영화
4,고양이에게 밥을 주지 마세요,10,길고양이들과 공생하는건 좋은일이라고생각합니다
...,...,...,...
9995,범죄도시2,10,재미집니다. 유머코드는 더 많이졌고요
9996,신비한 동물들과 덤블도어의 비밀,9,호그와트 많이 나와서 해덕은 그저 눈물만.... 솔직히 스토리 진짜...ㅋㅋ 너무 ...
9997,범죄도시2,10,시간가는줄 모르고 봤네요~단 한 순간도 화면에서 눈을 뗄 수 없을 정도로 몰입감 최...
9998,매스,7,NaN


In [3]:
data = pd.DataFrame()
data['X']=df['리뷰']
data['y']=df['평점']
data

,X,y
0,NaN,10
1,내누군지아니 나 장첸이야~!! ㅋ,10
2,시원시원한 타격감 ㅋㅋㅋ 청량하다 마동석,10
3,최고의 마동석표 웰메이드 액션 코믹 영화,10
4,길고양이들과 공생하는건 좋은일이라고생각합니다,10
...,...,...
9995,재미집니다. 유머코드는 더 많이졌고요,10
9996,호그와트 많이 나와서 해덕은 그저 눈물만.... 솔직히 스토리 진짜...ㅋㅋ 너무 ...,9
9997,시간가는줄 모르고 봤네요~단 한 순간도 화면에서 눈을 뗄 수 없을 정도로 몰입감 최...,10
9998,NaN,7


In [4]:
s_w=set(['은','는','이','가','를','들','에게','의','을','도','으로','만','라서','하다'])

In [5]:
data.isnull().values.any()

True

In [6]:
data['X'].nunique(), data['y'].nunique()

(9258, 10)

In [7]:
data=data.drop_duplicates(subset=['X'])
data['clean_X']=data.X.str.replace('[^ㄱ-ㅎㅏ-ㅣ가-힣 ]','')
data['clean_X']=data.clean_X.str.replace('^ +','')
data['clean_X']=data.clean_X.replace('',np.nan)
data=data.dropna(how='any')

C:\Users\user\AppData\Local\Temp/ipykernel_24444/3476282902.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  data['clean_X']=data.X.str.replace('[^ㄱ-ㅎㅏ-ㅣ가-힣 ]','')
C:\Users\user\AppData\Local\Temp/ipykernel_24444/3476282902.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['clean_X']=data.X.str.replace('[^ㄱ-ㅎㅏ-ㅣ가-힣 ]','')
C:\Users\user\AppData\Local\Temp/ipykernel_24444/3476282902.py:3: FutureWarning: The default value of regex will change from True to False in a future version.
  data['clean_X']=data.clean_X.str.replace('^ +','')
C:\Users\user\AppData\Local\Temp/ipykernel_24444/3476282902.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
T

In [8]:
okt=Okt()
from tqdm import tqdm 
X_data=[] 
for i in tqdm(data['clean_X']): 
    tk_d=okt.morphs(i) 
    end_d=[w for w in tk_d if not w in s_w]
    X_data.append(' '.join(end_d))

100%|██████████████████████████████████████████████████████████████████████████████████| 9234/9234 [00:59<00:00, 154.64it/s]


In [9]:
data['y_name']=data['y']
data['encoder_y']=LabelEncoder().fit_transform(data['y'])
data['categorical_y']=list(to_categorical(data['encoder_y']))

In [29]:
Y=to_categorical(data['encoder_y']) #다중
#Y=np.array(data['encoder_y'])#이진

In [30]:
Y.shape

(9234, 10)

In [31]:
X=np.array(X_data)

In [32]:
x_data,test_x,y_data,test_y = train_test_split(X,Y,test_size=0.3,random_state=0)

In [33]:
train_x,val_x,train_y,val_y = train_test_split(x_data,y_data,test_size=0.2,random_state=0)

In [34]:
tk = Tokenizer()
tk.fit_on_texts(train_x)

In [35]:
n=len([d for d in sorted(list(tk.word_counts.items()),key=lambda x:x[1]) if d[1]>4])+1

In [36]:
n

1966

In [37]:
token=Tokenizer(n)
token.fit_on_texts(train_x)

In [38]:
token_train_x=token.texts_to_sequences(train_x)
token_test_x=token.texts_to_sequences(test_x)
token_val_x=token.texts_to_sequences(val_x)

In [39]:
drop_train = [index for index, sentence in enumerate(token_train_x) if len(sentence) < 1]
drop_test = [index for index, sentence in enumerate(token_test_x) if len(sentence) < 1]
drop_val = [index for index, sentence in enumerate(token_val_x) if len(sentence) < 1]

In [40]:
token_train_x = np.delete(token_train_x, drop_train, axis=0)
train_y = np.delete(train_y, drop_train, axis=0)
token_test_x = np.delete(token_test_x, drop_test, axis=0)
test_y = np.delete(test_y, drop_test, axis=0)
token_val_x = np.delete(token_val_x, drop_val, axis=0)
val_y = np.delete(val_y, drop_val, axis=0)

In [41]:
w_l=len(pad_sequences(token_train_x)[0])

In [42]:
w_l

188

In [43]:
train_inputs = pad_sequences(token_train_x,maxlen=w_l)
test_inputs = pad_sequences(token_test_x,maxlen=w_l)
val_inputs = pad_sequences(token_val_x,maxlen=w_l)

In [44]:
train_outputs=train_y
test_outputs=test_y
val_outputs=val_y

In [45]:
전처리_된_데이터={}
전처리_된_데이터['학습_데이터']=train_inputs
전처리_된_데이터['학습_결과']=train_outputs
전처리_된_데이터['태스트_데이터']=test_inputs
전처리_된_데이터['태스트_결과']=test_outputs
전처리_된_데이터['검증_데이터']=val_inputs
전처리_된_데이터['검증_결과']=val_outputs
전처리_된_데이터['토큰']=token
전처리_된_데이터['단어수']=n
전처리_된_데이터['불용어']=s_w
전처리_된_데이터['입력길이']=w_l

In [46]:
전처리_된_데이터.keys()

dict_keys(['학습_데이터', '학습_결과', '태스트_데이터', '태스트_결과', '검증_데이터', '검증_결과', '토큰', '단어수', '불용어', '입력길이'])

In [47]:
import pickle
with open("data.pickle","wb") as fw:
    pickle.dump(전처리_된_데이터, fw)